In [3]:
import panda_dataset as pds
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader

from PIL import Image
from IPython import display

In [4]:
def as_gif(images, path="sample.gif"):
    images[0].save(path, save_all=True, append_images=images, duration=int(1000/30), loop=0)
    gif_bytes = open(path,"rb").read()
    return gif_bytes

In [5]:
def panda_custom_collate(batch):
    collated_batch = {}
    collated_intrinsic = torch.tensor([])
    collated_extrinsic = torch.tensor([])
    collated_timestamps = []
    collated_steps = []
    for item in batch:
        collated_intrinsic = torch.concatenate((collated_intrinsic, torch.tensor(item["calibration_info"]["intrinsic"])))
        collated_extrinsic = torch.concatenate((collated_extrinsic, torch.tensor(item["calibration_info"]["extrinsic"])))
        collated_timestamps.append(item["timestamps"])
        collated_steps.append(item["steps"])

    collated_batch["calibration_info"] = dict()
    collated_batch["calibration_info"]["intrinsic"] = collated_intrinsic
    collated_batch["calibration_info"]["extrinsic"] = collated_extrinsic
    collated_batch["timestamps"] = collated_timestamps
    collated_batch["steps"] = collated_steps
    
    return collated_batch

In [6]:
panda_dataset = pds.PandaDataset(root_dir="./panda_dataset", transform=pds.ToTensor())
dataloader = DataLoader(panda_dataset, batch_size=3, num_workers=5, collate_fn=panda_custom_collate, shuffle=False)
for i_batch, sample_batched in enumerate(dataloader):
    images = []
    timestamps = sample_batched["timestamps"]
    print(timestamps)
    for i, timestamp in enumerate(sample_batched["steps"]):
        print(len(sample_batched["steps"]))

[['1723856436.7055814', '1723856436.7390432', '1723856436.7723656', '1723856436.8100057', '1723856436.8390346', '1723856436.8723335', '1723856436.9057052', '1723856436.9390254', '1723856436.9723885', '1723856437.005711', '1723856437.0390317', '1723856437.0723436', '1723856437.1056807', '1723856437.138987', '1723856437.172354', '1723856437.2057052', '1723856437.2390285', '1723856437.2723649', '1723856437.3056931', '1723856437.3390074', '1723856437.3723714', '1723856437.4057062', '1723856437.4390292', '1723856437.47237', '1723856437.505703', '1723856437.5390162', '1723856437.5723464', '1723856437.6057034', '1723856437.6390305', '1723856437.6723392', '1723856437.705683', '1723856437.739034', '1723856437.7723897', '1723856437.805669', '1723856437.8390324', '1723856437.8723636', '1723856437.9057019', '1723856437.939007', '1723856437.9723656', '1723856438.0056913', '1723856438.0390038', '1723856438.072367', '1723856438.1057022', '1723856438.1390047', '1723856438.1723812', '1723856438.2057135

In [8]:
# Prints camera intrinsics and extrinsics and outputs a gif for each episode
panda_dataset = pds.PandaDataset(root_dir="./panda_dataset", transform=pds.ToTensor())
dataloader = DataLoader(panda_dataset, batch_size=1, num_workers=0)
for i_batch, sample_batched in enumerate(dataloader):
    images = []
    timestamps = sample_batched["timestamps"]
    cam_in = sample_batched["calibration_info"]["intrinsic"][0]
    cam_ex = sample_batched["calibration_info"]["extrinsic"][0]
    print(f"Camera Intrinsics:\n{cam_in}")
    print(f"Camera Extrinsics:\n{cam_ex}")
    for i, timestamp in enumerate(sample_batched["steps"]):
        step = sample_batched["steps"][timestamp]
        image = step["img_data"].squeeze().permute(1,2,0).numpy()
        images.append(Image.fromarray(image))
    ep_num = str(i_batch+1).zfill(4)
    as_gif(images, path=f"./assets/gifs/panda_{ep_num}.gif")


Camera Intrinsics:
tensor([[151.1343,   0.0000, 160.0976],
        [  0.0000, 151.1347, 120.6547],
        [  0.0000,   0.0000,   1.0000]], dtype=torch.float64)
Camera Extrinsics:
tensor([[ 0.8729, -2.1360,  1.4145,  0.5159,  0.4621,  0.3596]],
       dtype=torch.float64)
Camera Intrinsics:
tensor([[151.1343,   0.0000, 160.0976],
        [  0.0000, 151.1347, 120.6547],
        [  0.0000,   0.0000,   1.0000]], dtype=torch.float64)
Camera Extrinsics:
tensor([[ 0.8729, -2.1360,  1.4145,  0.5159,  0.4621,  0.3596]],
       dtype=torch.float64)
Camera Intrinsics:
tensor([[151.1343,   0.0000, 160.0976],
        [  0.0000, 151.1347, 120.6547],
        [  0.0000,   0.0000,   1.0000]], dtype=torch.float64)
Camera Extrinsics:
tensor([[ 0.8729, -2.1360,  1.4145,  0.5159,  0.4621,  0.3596]],
       dtype=torch.float64)
Camera Intrinsics:
tensor([[151.1343,   0.0000, 160.0976],
        [  0.0000, 151.1347, 120.6547],
        [  0.0000,   0.0000,   1.0000]], dtype=torch.float64)
Camera Extrinsics:
t